In [ ]:
import uproot
import os
import pickle
import coffea
from coffea import nanoevents
from coffea.lookup_tools.dense_lookup import dense_lookup
import numpy as np
import awkward as ak
import pandas as pd

In [ ]:
MAIN_DIR = "../../../"

plot_dir = f"{MAIN_DIR}/plots/v11v12Checks/23Nov10"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
paths = {
    "v11": {
        "qcd": "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/Run3Summer22EENanoAODv11/QCD-4Jets_HT-2000_TuneCP5_13p6TeV_madgraphMLM-pythia8/NANOAODSIM/126X_mcRun3_2022_realistic_postEE_v1-v2/2810000/02d3ed0c-74c1-464d-bff9-8345ae4a6dd5.root",
        "hh4b": "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/Run3Summer22EENanoAODv10/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_124X_mcRun3_2022_realistic_postEE_v1-v2/2540000/46c288ba-4f36-450b-9108-8070eca82d95.root",
        "data": "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2022F/JetMET/NANOAOD/PromptNanoAODv11_v1-v2/2540000/02b8ddba-9103-4801-bb74-cab4d1fcc6ca.root"
    },
    "v12": {
        "qcd": "root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/QCD-4Jets_HT-2000_TuneCP5_13p6TeV_madgraphMLM-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v2/2520000/05e001cc-8902-4fa0-b706-98fc89013dc2.root",
        "hh4b": 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/2540000/00d98799-ada3-4a26-8558-5052891a8d23.root',
        "data": 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2022E/JetMET/NANOAOD/22Sep2023-v1/30000/002cde4c-efa1-430d-b651-61b6dff4d208.root'
    }
}

In [ ]:
events_dict = {
    v: {
        key: nanoevents.NanoEventsFactory.from_root(
                 path, schemaclass=nanoevents.NanoAODSchema
             ).events() for key, path in paths[v].items()
    } for v in paths
}

In [ ]:
import sys

In [ ]:
sys.getsizeof(events_dict["v11"]["qcd"])

In [ ]:
ak.count(events_dict["v11"]["qcd"].Muon.pt, axis=1)

In [ ]:
plt.hist(ak.count(events_dict["v11"]["qcd"][:157660].Muon.pt, axis=1), histtype='step')
plt.hist(ak.count(events_dict["v12"]["qcd"][:157660].Muon.pt, axis=1), histtype='step')

In [ ]:
qcdv11_events.FatJet.pt

In [ ]:
qcdv12_events.FatJet.pt

In [ ]:
import matplotlib.pyplot as plt

bins = np.linspace(0, 2000, 31)
plt.hist(qcdv11_events.FatJet.pt[:, 0:1], bins=bins, histtype='step', density=True)
plt.hist(qcdv12_events.FatJet.pt[:, 0:1], bins=bins, histtype='step', density=True)
plt.show()

In [ ]:
bins = np.linspace(0, 300, 31)
plt.hist(qcdv11_events.FatJet.msoftdrop[:, 0:1], bins=bins, histtype='step', density=True)
plt.hist(qcdv12_events.FatJet.msoftdrop[:, 0:1], bins=bins, histtype='step', density=True)
plt.show()

In [ ]:
events_dict

In [ ]:
pnet_masses = {"v11": {key: events.FatJet.particleNet_mass[:, 0:1] for key, events in events_dict["v11"].items()}}
pnet_masses["v12 JEC Mass"] = {
    key: (events.FatJet.mass * events.FatJet.particleNet_massCorr)[:, 0:1] for key, events in events_dict["v12"].items()
}
pnet_masses["v12 Raw Mass"] = {
    key: (events.FatJet.mass * (1 - events.FatJet.rawFactor) * events.FatJet.particleNet_massCorr)[:, 0:1] for key, events in events_dict["v12"].items()
}

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

hep.style.use("CMS")

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams["lines.linewidth"] = 2
plt.rcParams["grid.color"] = "#CCCCCC"
plt.rcParams["grid.linewidth"] = 0.5
plt.rcParams["figure.edgecolor"] = "none"

labels = {
    "qcd": "QCD-HT2000",
    "hh4b": "ggF HH4b",
    "data": "Data",
}

In [ ]:
bins = np.linspace(0, 300, 31)

for key in pnet_masses["v11"]:
    bins = np.linspace(0, 300, 31) if key == "qcd" else np.linspace(0, 200, 31)
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    plt.rcParams.update({"font.size": 24})
    for v, samples in pnet_masses.items():
        masses = samples[key]
        ax.hist(ak.flatten(masses), bins=bins, histtype='step', density=True, label=v)

    ax.set_title(labels[key], x=0.45)
    ax.legend()
    hep.cms.label(data=True, label="Internal", year="2022EE", lumi="21")
    plt.savefig(f"{plot_dir}/{key}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
bins = np.linspace(0, 300, 31)
plt.hist(qcdv11_events.FatJet.particleNet_mass[:, 0:1], bins=bins, histtype='step', density=True, label="v11")
plt.hist((qcdv12_events.FatJet.mass * qcdv12_events.FatJet.particleNet_massCorr)[:, 0:1], bins=bins, histtype='step', density=True, label="v12")
plt.xlabel("ParticleNet Mass")
plt.legend()
plt.show()